In [1]:
from pathlib import Path
from tqdm.auto import tqdm
import shutil
import pandas as pd
import ray
from toolkit_run.ray.server import LabRayToolkitServer
import json
from collections import defaultdict

In [2]:
server = LabRayToolkitServer()
server.dashboard_url

'https://327b68c6-51de-4024-bef7-0afee15d5f68-8000.job.console.elementai.com'

In [3]:
server.scale_cluster(60)

'OK'

In [9]:
server.shutdown()

## Read 1K meta data

In [3]:
the_stack_meta_path = Path('/data/hf_repos/the-stack-metadata')
the_stack_path = Path('/data/hf_repos/the-stack-v1.1')

In [4]:
with open('/data/hf_repos/the-stak-repo-level/meta_data/java/1K_repo_2_bucket.json', 'rt') as f:
    repo_2_bucket_1K = json.load(f)

In [5]:
with open('/data/hf_repos/the-stak-repo-level/meta_data/java/1K_repos_commit_hashes.json', 'rt') as f:
    repos_commit_hashes_1K = json.load(f)

In [6]:
repos_commit_hashes_dst_buckets_1K = [(el[0], el[1], repo_2_bucket_1K[el[0]]) for el in repos_commit_hashes_1K]

In [7]:
df_repos_commit_hashes_dst_buckets_1K = pd.DataFrame(
    data=repos_commit_hashes_dst_buckets_1K, columns=['name', 'head_hexsha', 'dst_bucket']
)

In [8]:
df_repos_commit_hashes_dst_buckets_1K

,name,head_hexsha,dst_bucket
0,WendyOmollo/CaesarCipher,399d3aa6057ef43ac86eea5f4992085bd045a8fd,250
1,oktaykcr/bom-app-be,91df70f566814b32e3f1923547f4edadf88352cc,650
2,abbashus/sql,d81ef73c968b2ae7a9f52e362de294d4dec8e855,265
3,mryan/perl6-nqp,687e2f12c866ee1498229ac1ac234ff56b7d02b8,623
4,jMonkeyEngine-mirrors/iso-terrain-jayfella-github,7da9fb3d65de5e8ecfd6f1bce16b16f39de236ee,498
...,...,...,...
995,elsadeny/VideoEnabledWebView-1,74c2c56019016af7c0aba6e3cf43cddb940ba4ac,411
996,TengJunWei/butterknife,7c39e473efd501269a444f2e3d03184eb41d2d05,230
997,ModificationStation/StationAPI,a359af0c8e7f4467e56a5db78620b0b910f45a40,161
998,Mr-Frimman/Apotheosis-Fabric,ed2817ae54af41c99c927bfb2d8c1ce0132a8c0d,163


In [9]:
file_list_1k = pd.read_parquet('/data/hf_repos/the-stak-repo-level/meta_data/java/1K_file_list.parquet')

In [12]:
file_list_1k

,hexsha,path,name,head_hexsha,dst_bucket
388123,6a8fa90325c6fe21399be8c5d4514c099eddac05,src/main/java/n10s/skos/load/SkosImporter.java,dpb587/neosemantics,c968e28aca8f2f327a90dfeefac63756e19190bd,398.0
388124,0df86c5511198a7fb84bf270cab07fb5cf435893,src/main/java/n10s/result/GraphResult.java,dpb587/neosemantics,c968e28aca8f2f327a90dfeefac63756e19190bd,398.0
388125,4ce9ae926792bcfd8003e9e305b7f7e5dfc22eb5,src/main/java/n10s/result/NamespacePrefixesRes...,dpb587/neosemantics,c968e28aca8f2f327a90dfeefac63756e19190bd,398.0
388126,36f21d3259ae0961abcd953927dfee72c0c1c15a,src/main/java/n10s/result/NodeResult.java,dpb587/neosemantics,c968e28aca8f2f327a90dfeefac63756e19190bd,398.0
388127,ef7f02a529f48221ccef4c086e9f6e2da872ed5e,src/main/java/n10s/result/StreamedStatement.java,dpb587/neosemantics,c968e28aca8f2f327a90dfeefac63756e19190bd,398.0
...,...,...,...,...,...
212778,44b49a2073a14a2bb55ad0d9e38ccf608f2fc0c8,src/main/java/org/eclipse/microprofile/starter...,jim-krueger/microprofile-starter,8b7d80d52e61cb165719b165ba9305f6e4fe769c,514.0
212779,cc01b965f7cc8b1679569f7bd4eb87b1b3822f34,src/main/java/org/eclipse/microprofile/starter...,jim-krueger/microprofile-starter,8b7d80d52e61cb165719b165ba9305f6e4fe769c,514.0
212780,c608c7edcd1d0c99749638c147d80f6a392274ab,src/main/java/org/eclipse/microprofile/starter...,jim-krueger/microprofile-starter,8b7d80d52e61cb165719b165ba9305f6e4fe769c,514.0
212781,ca0a8134b4062cf041451573d6db7e26e27cb3c8,src/main/java/org/eclipse/microprofile/starter...,jim-krueger/microprofile-starter,8b7d80d52e61cb165719b165ba9305f6e4fe769c,514.0


In [16]:
file_list_1k[['name', 'head_hexsha', 'hexsha']].groupby(['name', 'head_hexsha']).count().sort_values('hexsha')

,,hexsha
name,head_hexsha,
01010101lzy/issual,9cdc972668fcc5aa4a6e8b4de72cfec92710248b,1
nikolvas/ads_y2021,26912cfd73a7ebfce9c3c3ac42648667258abd35,1
ZZL18/zzl,73131f4fccf33ae471100b0adacd35067cc56c73,1
nurdiansyah/SveltePress,832151b2ecc7cd009d5f8ad368aab1bb1feb4aab,1
oke-ya/CCNativeExtensionTemplate,413c04a3531543766c1e3ff51938134ffa16f43a,1
...,...,...
smouelhi/tutorials,8a5d9843f6e7ee86a8f7bb142f3dfce5f47d1068,7706
slhmy/trino,5e4760102ce500ca4bd102a370bcb4fbf8812809,8281
Diffblue-benchmarks/Elastic-elasticsearch,93f3d12c759f63c12f6e773cfc514345627a4c16,11097


In [19]:
file_list_1k[file_list_1k['name'] == 'oke-ya/CCNativeExtensionTemplate']

,hexsha,path,name,head_hexsha,dst_bucket
138197,1e1d32c0dcafb04ee9dcc03b7a5eb4a693c5d025,proj.android/java/src/com/oke_ya/CCNativeExten...,oke-ya/CCNativeExtensionTemplate,413c04a3531543766c1e3ff51938134ffa16f43a,650.0


## Test results

In [4]:
missing = []
dst_root = Path('/data/hf_repos/the-stak-repo-level/data/java')
for i, row_file_list in tqdm(file_list_1k.iterrows(), total=len(file_list)):
    dst_path = (
        dst_root / str(int(row_file_list['dst_bucket'])) / row_file_list['name'] /
        row_file_list['head_hexsha'] /  row_file_list['path']
    )
    if not dst_path.is_file():
        missing.append(dst_path)

  0%|          | 0/232407 [00:00<?, ?it/s]

In [5]:
len(missing)

246

In [54]:
lic = pd.read_parquet('/dataset/repositories_zipped2_logs/indexes/repo_info/df_2022-01-24_all_licenses_no_vanity/lic/data-0.parquet')
ri = pd.read_parquet('/dataset/repositories_zipped2_logs/indexes/repo_info/df_2022-01-24_all_licenses_no_vanity/ri/data-0.parquet')
fi = pd.read_parquet('/dataset/repositories_zipped2_logs/indexes/repo_info/df_2022-01-24_all_licenses_no_vanity/fi/data-0.parquet')

In [47]:
lic['ri_id'].nunique()

2044

In [55]:
lic.loc[lic['file'].isna(), 'file'] = 'None'
idx = lic.groupby(["ri_id", 'file'])['confidence'].transform(max) == lic['confidence']
lic1 = lic[idx]
lic1 = lic1.groupby('ri_id').apply(lambda x: list(set(x['license']))).to_frame('licenses')


In [56]:
ri1 = ri.merge(lic1, left_on='id', right_on='ri_id', how='right')

In [57]:
fi = fi[fi['lang_ex'] == 'Java']
fi1 = fi.merge(ri1, left_on='ri_id', right_on='id', how='inner')

In [62]:
assert fi1['ri_id'].nunique() == fi['ri_id'].nunique()

In [63]:
fi['ri_id'].nunique()

187